<p><span style="background-color: #ccffcc;"><strong>This SAGE notebook contains a simple version of the Fincke-Pohst algorithm, thanks to which the theta series of lattices can be computed.</strong></span></p>
<p><span style="background-color: #ccffcc;"><strong>The algorithm is pretty basic, and works for small dimensions (tested up to dimensions 12).</strong></span></p>
<p><span style="background-color: #ccffcc;"><strong>Then several theta series are computed.</strong></span></p>

<p>To enumerate lattice points, we implement Fincke-Pohst.</p>
<p>The reference is the chapter 10 of the book</p>
<h1 class="bookTitle"><span style="font-size: small;">Lattice Basis Reduction: An Introduction to the LLL Algorithm and Its Applications</span></h1>
<p>by Murray R. Bremner</p>
<p>The output uses the modular form convention, that is, $q^{i/2}$ instead of $q^i$. </p>

In [1]:
def FP(G,C):
    #Fincke-Pohst
    #given a Gram matrix G and an upper bound C on the squared norm, enumerate lattice points
    #output: obt_result = list of integer vectors u such that u*B is a lattice point
    #        vec_norm   = list containing the corresponding norms
    #        list_norm  = contains the distinct norms appearing and their multiplicity 
    #        st = theta series
    #*******************************************************************************************
    #modify the output of the existing cholesky to have a decomposition of the form U^T D U
    L = G.cholesky()
    n = G.nrows()
    D = matrix.zero(RR,n)
    U = matrix(RR,n)
    for i in range(0,n):
        D[i,i] = L[i,i]^2
        U[i,:] = (L[:,i]/L[i,i]).transpose()
    #initialize a list that will contain the lattice points obtained
    #it contains one element, the empty set
    obt_result = [[]]
    for k in range(n-1,-1,-1):
        #initialize a list to contain new lattice point coordinates
        new_result = []
        for r in range(0,len(obt_result)):
            #load the partial coordinates already obtained
            #does padding at the beginning to tally with indices when computing S and T
            xvalue = flatten([range(0,k+1),obt_result[r]])
            S = sum(D[i,i]*(xvalue[i]+sum(U[i,j]*xvalue[j] for j in range(i+1,n)))**2 for i in range(k+1,n,+1))
            T = sum(U[k,j]*xvalue[j] for j in range(k+1,n))
            #dirty way to get rid of numerical approximations of the type -1e-15
            if C-S < 0:
                S = C
            #had to introduce a truncation so that ceil(2.0000000000?)=2 and not 3
            low_bound = ceil((-sqrt((C-S)/D[k,k])-T).n(digits=10))
            up_bound = floor((sqrt((C-S)/D[k,k])-T).n(digits=10))
            for x in range(low_bound,up_bound+1):
                #create a new partial vector by adding the coordinate found
                xr = flatten([x,obt_result[r]])
                #look for the last entry which is not zero
                mm = -1
                for m in range(0,len(xr)):
                    if xr[m] <> 0:
                        mm = m
                if mm == -1:       
                    new_result.append(xr)
                else:
                    if xr[mm] > 0:
                        new_result.append(xr)
        obt_result = new_result
    #this list stores norms indexed with the corresponding vectors
    vec_norm = []
    for i in range(0,len(obt_result)):
        vec_norm.append(round((vector(obt_result[i])*L).norm()**2))
    vec_norm.sort()
    #this list serves to count the no of different norms
    list_norm = []
    list_norm.append([vec_norm[0],vec_norm.count(vec_norm[0])])
    #counts no of occurences of the norm
    for i in range(1,len(vec_norm)):
        if (vec_norm[i] in vec_norm[0:i]) == false:
            #factor of 2, since for every nonzero vector v of given norm, -v has the same norm
            list_norm.append([vec_norm[i],2*vec_norm.count(vec_norm[i])])
    #generates the theta series
    R.<q> = PowerSeriesRing(ZZ, default_prec=10)
    st = 1
    for i in range(1,len(list_norm)):
          st = st + list_norm[i][1]*q**(round(list_norm[i][0]/2))
    st = st +O(q**(round(list_norm[i][0]/2)+1))
    return obt_result, vec_norm, list_norm, st

<p>We try the book example.</p>

In [2]:
B = matrix(ZZ, [[1,-4,9,-7,9],[-3,2,0,5,-5],[9,-8,-1,-4,-5],[6,6,-3,-8,8],[4,3,-6,5,-8]])
G = B.transpose()*B
v=FP(G,80)
v[0]

[[0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0],
 [1, 1, 1, 1, 0],
 [0, -1, -1, 1, 1],
 [1, 0, 0, 1, 1],
 [1, 0, 0, 2, 1],
 [1, 0, 1, 2, 1],
 [2, 0, 1, 2, 1],
 [1, -1, -1, 2, 2],
 [2, -1, 0, 3, 2],
 [1, -2, -2, 3, 3],
 [2, -1, 0, 4, 3],
 [2, -2, -1, 5, 4]]

<p><span style="text-decoration: underline; font-size: x-large;">Some Known Examples</span></p>
<p>We try a few known theta series.</p>
<p>The theta series of QQF.4.a starts with  <tt>1, 6, 18, 24, 42, 6, 72, 48, 90, 78, 18, 72...</tt></p>

In [12]:
GQQF4a = matrix(ZZ,[[2,1,0,0],[1,2,1,0],[0,1,4,5],[0,0,5,10]])
vQQF4a = FP(GQQF4a,10)
vQQF4a[3]

1 + 6*q + 18*q^2 + 24*q^3 + 42*q^4 + 6*q^5 + O(q^6)

In [37]:
GMaas = matrix(ZZ, [[4,-2,1,1,-1,-2,0,2],
[-2, 4, -1, 0, -1, 1, 1, -1],
[1 ,-1, 4, -1, 1, -2 ,-2 ,0],
[1, 0 ,-1, 4, 1, 1, 2, 1],
[-1, -1, 1, 1, 4, 1 ,-1 ,-2],
[-2, 1, -2, 1, 1, 4 ,2 ,0],
[0, 1, -2, 2, -1, 2, 4, 1],
[2 ,-1, 0, 1, -2, 0, 1, 4]])
vMaas = FP(GMaas,10)
vMaas[3]

1 + 120*q^2 + 240*q^3 + 600*q^4 + 1312*q^5 + O(q^6)

<p>The theta series of $A_2$ starts with <tt> <tt>1, 6, 0, 6, 6, 0, 0, 12, 0, 6, 0, 0, 6, 12</tt>,...</tt></p>

In [13]:
GA2 = matrix(ZZ,[[2,1],[1,2]])
vA2 = FP(GA2,31)
vA2[3]

1 + 6*q + 6*q^3 + 6*q^4 + 12*q^7 + 6*q^9 + 6*q^12 + 12*q^13 + O(q^14)

<p>The theta series of $D_4$.</p>

In [19]:
GD4 = matrix(ZZ,[[2,1,0,0],[1,2,1,1],[0,1,2,0],[0,1,0,2]])
vD4 = FP(GD4,15)
vD4[3]

1 + 24*q + 24*q^2 + 96*q^3 + 24*q^4 + 144*q^5 + 96*q^6 + 192*q^7 + O(q^8)

<p><span style="text-decoration: underline; font-size: x-large;">Some Modular Lattices</span></p>
<p>The theta series of the even extremal 7-modular lattice in dimension 2.</p>

In [21]:
GL7 = matrix(ZZ,[[2,1],[1,4]])
vL7 = FP(GL7,45)
vL7[3]

1 + 2*q + 4*q^2 + 6*q^4 + 2*q^7 + 8*q^8 + 2*q^9 + 4*q^11 + 4*q^14 + 10*q^16 + 4*q^18 + 8*q^22 + O(q^23)

<p>The theta series of the even extremal 11-modular lattice in dimension 2.</p>

In [23]:
GL11 = matrix(ZZ,[[2,1],[1,6]])
vL11 = FP(GL11,31)
vL11[3]

1 + 2*q + 4*q^3 + 2*q^4 + 4*q^5 + 6*q^9 + 2*q^11 + 4*q^12 + 8*q^15 + O(q^16)

<p>The theta series of the even extremal 23-modular lattice in dimension 2.</p>

In [25]:
GL23 = matrix(ZZ,[[4,1],[1,6]])
vL23 = FP(GL23,40)
vL23[3]

1 + 2*q^2 + 2*q^3 + 2*q^4 + 2*q^6 + 2*q^8 + 2*q^9 + 4*q^12 + 2*q^13 + 4*q^16 + 4*q^18 + O(q^19)

<p>We also compute the theta series of the other even 23-modular lattice in dimension 2 which is not extremal.</p>

In [27]:
GL23_2 = matrix(ZZ,[[2,1],[1,12]])
vL23_2 = FP(GL23_2,40)
vL23_2[3]

1 + 2*q + 2*q^4 + 4*q^6 + 4*q^8 + 2*q^9 + 4*q^12 + 2*q^16 + 4*q^18 + O(q^19)

<p>The theta series of QQF.4.g, an even strongly 6-modular extremal lattice in dimension 4.</p>

In [29]:
Gqqf4g = matrix(ZZ,[[2,1,0,0],[1,2,0,0],[0,0,4,2],[0,0,2,4]])
vqqf4g = FP(Gqqf4g,15)
vqqf4g[3]

1 + 6*q + 6*q^2 + 42*q^3 + 6*q^4 + 36*q^5 + 42*q^6 + 48*q^7 + O(q^8)

<p>The theta series of QQF.4.i, the other even strongly 6-modular extremal lattice in dimension 4.</p>

In [31]:
Gqqf4i = matrix(ZZ,[[2,0,1,1],[0,2,1,1],[1,1,4,1],[1,1,1,4]])
vqqf4i = FP(Gqqf4i,15)
vqqf4i[3]

1 + 4*q + 20*q^2 + 4*q^3 + 52*q^4 + 24*q^5 + 20*q^6 + 32*q^7 + O(q^8)

<p>The theta series of a 4-dimensional 11-modular lattice which turns out to be extremal.</p>

In [38]:
G11_dim4 = matrix(ZZ, [[4,2,0,-1],[2,4,-1,0],[0,-1,4,-2],[-1,0,-2,4]])
vG11_dim4 = FP(G11_dim4,15)
vG11_dim4[3]

1 + 12*q^2 + 12*q^3 + 12*q^4 + 12*q^5 + 24*q^6 + 24*q^7 + O(q^8)

<p>The theta series of a 12-dimensional  11-modular lattice which turns out to be extremal.</p>

In [41]:
G11_dim12 = matrix(ZZ, [[ 6 ,-2, -2,  2,  2,  0, -2,  0, -2,  1, -2,  0],      
[-2,  6,  2,  0,  2 , 2,  2,  2,  3,  2,  2,  0],      
[-2,  2,  6,  2, -2 , 2,  3, -2,  0,  2,  0, -2],      
[ 2,  0,  2,  6, -2 , 2,  2, -2, -2,  2, -2,  1],      
[ 2,  2, -2, -2,  8 , 2, -2,  4,  1, -1, -1, -1],      
[ 0,  2,  2,  2,  2 , 6,  0,  1, -1,  0, -2, -2],      
[-2,  2,  3,  2, -2 , 0,  6, -2,  0,  0,  0, -1],      
[ 0,  2, -2, -2,  4 , 1, -2,  6,  3,  0,  2,  0],      
[-2,  3,  0, -2,  1 ,-1,  0,  3,  6,  0,  2,  0],      
[ 1,  2,  2,  2, -1 , 0,  0,  0,  0,  6,  2,  2],      
[-2,  2,  0, -2, -1 ,-2,  0,  2,  2,  2,  6,  2],      
[ 0,  0, -2,  1, -1 ,-2, -1,  0,  0,  2,  2,  6]])
vG11_dim12 = FP(G11_dim12,15)
vG11_dim12[3]

1 + 100*q^3 + 444*q^4 + 1188*q^5 + 2908*q^6 + 6504*q^7 + O(q^8)

<p><span style="text-decoration: underline; font-size: x-large;">15-modular Lattices</span></p>
<p>Then we compute the theta series of the three 15-modular lattices which are strongly modular.</p>

In [14]:
G1 = matrix(ZZ,[[2,1,0,0],[1,2,0,0],[0,0,10,5],[0,0,5,10]])
v1 = FP(G1,31)
print det(G1),v1[3]

225 1 + 6*q + 6*q^3 + 6*q^4 + 6*q^5 + 36*q^6 + 12*q^7 + 36*q^8 + 42*q^9 + 78*q^12 + 12*q^13 + 36*q^14 + 6*q^15 + O(q^16)

In [15]:
G2 = matrix(ZZ,[[2,0,0,1],[0,4,1,0],[0,1,4,0],[1,0,0,8]])
v2 = FP(G2,31)
print det(G2),v2[3]

225 1 + 2*q + 4*q^2 + 10*q^3 + 10*q^4 + 2*q^5 + 32*q^6 + 12*q^7 + 24*q^8 + 38*q^9 + 4*q^10 + 16*q^11 + 74*q^12 + 20*q^13 + 36*q^14 + 10*q^15 + O(q^16)

In [16]:
G3 = matrix(ZZ,[[4,2,1,1],[2,4,-1,2],[1,-1,6,2],[1,2,2,6]])
v3 = FP(G3,40)
print det(G3),v3[3]

225 1 + 6*q^2 + 12*q^3 + 12*q^4 + 30*q^6 + 12*q^7 + 18*q^8 + 36*q^9 + 6*q^10 + 24*q^11 + 72*q^12 + 24*q^13 + 36*q^14 + 12*q^15 + 48*q^16 + 24*q^17 + 114*q^18 + 24*q^19 + 12*q^20 + O(q^21)

<p>We recognize the theta series of E(15).</p>

<p><span style="text-decoration: underline; font-size: x-large;">14-modular Lattices</span></p>
<p>We compute the theta series of the three 14-modular lattices which are strongly modular.</p>

In [32]:
G4 = matrix(ZZ,[[2,1,0,0],[1,4,0,0],[0,0,4,2],[0,0,2,8]])
v4 = FP(G4,15)
print det(G4),v4[3]

196 1 + 2*q + 6*q^2 + 4*q^3 + 18*q^4 + 8*q^5 + 28*q^6 + 2*q^7 + O(q^8)

In [34]:
G5 = matrix(ZZ,[[2,0,1,1],[0,2,1,1],[1,1,8,1],[1,1,1,8]])
v5 = FP(G5,15)
print det(G5),v5[3]

196 1 + 4*q + 4*q^2 + 20*q^4 + 8*q^5 + 32*q^6 + 4*q^7 + O(q^8)

In [35]:
G6 = matrix(ZZ,[[4,1,1,0],[1,4,0,1],[1,0,4,-1],[0,1,-1,4]])
v6 = FP(G6,41)
print det(G6),v6[3]

196 1 + 8*q^2 + 8*q^3 + 16*q^4 + 8*q^5 + 24*q^6 + 40*q^8 + 16*q^9 + 40*q^10 + 16*q^11 + 72*q^12 + 24*q^13 + 8*q^14 + 32*q^15 + 80*q^16 + 16*q^17 + 88*q^18 + 24*q^19 + 104*q^20 + O(q^21)

<p>We recognize the theta series of E(14).</p>